In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning)

In [11]:
data=pd.read_csv('output_quora.csv')

In [13]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,1,1,66,57,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,4,1,51,88,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1,1,73,59,...,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0,1,1,50,65,...,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,3,1,76,39,...,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [15]:
data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'freq_qid1', 'freq_qid2', 'q1len', 'q2len', 'q1_n_words', 'q2_n_words',
       'word_common', 'word_total', 'word_share', 'freq_q1+q2', 'freq_q1-q2',
       'cwc_min', 'cwc_max', 'csc_min', 'csc_max', 'ctc_min', 'ctc_max',
       'last_word_eq', 'first_word_eq', 'abs_len_diff', 'mean_len',
       'token_set_ratio', 'token_sort_ratio', 'fuzz_ratio',
       'fuzz_partial_ratio', 'longest_substr_ratio'],
      dtype='object')

In [31]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [23]:
questions = list(data['question1'].astype(str)) + list(data['question2'].astype(str))
tfidf = TfidfVectorizer(lowercase=False,)
tfidf.fit_transform(questions)
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [25]:
glove_nlp=spacy.load('en_core_web_lg')
len(glove_nlp("man").vector),len(glove_nlp("boy").vector)

(300, 300)

In [37]:
def get_weight_vectors(data, column):
    vectors = []
    for sentence in tqdm(list(data[column])):
        if pd.isnull(sentence):
            # Handle missing values by appending a zero vector
            vectors.append(np.zeros(300))  # Assuming GloVe embeddings of size 300
            continue
        try:
            doc = glove_nlp(sentence)
            mean_vector = np.zeros([len(doc), 300])
            for word in doc:
                vector = word.vector
                try:
                    idf = word2tfidf[str(word)]
                except KeyError:
                    idf = 0
                mean_vector += vector * idf
            mean_vector = mean_vector.mean(axis=0)
            vectors.append(mean_vector)
        except Exception as e:
            print(f"Error processing sentence: {sentence}, Error: {e}")
            vectors.append(np.zeros(300))  # Append zero vector on failure

    return vectors



In [39]:
%%time
data['q1_feats_m'] = get_weight_vectors(data, 'question1')


100%|██████████████████████████████████| 404290/404290 [40:13<00:00, 167.50it/s]

CPU times: user 28min 16s, sys: 49.3 s, total: 29min 5s
Wall time: 40min 13s


In [41]:
%%time
data['q2_feats_m'] = get_weight_vectors(data, 'question2')


100%|██████████████████████████████████| 404290/404290 [31:39<00:00, 212.84it/s]

CPU times: user 30min 41s, sys: 24.7 s, total: 31min 5s
Wall time: 31min 39s


In [42]:
%%time
df_q1 = pd.DataFrame(data.q1_feats_m.values.tolist(), index= data.index)
df_q2 = pd.DataFrame(data.q2_feats_m.values.tolist(), index= data.index)

CPU times: user 1min 16s, sys: 10min 22s, total: 11min 39s
Wall time: 15min 57s


In [43]:
df_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-16.180508,74.007053,-262.828838,-21.173333,153.037469,50.527348,-70.398460,167.927940,-254.153506,-0.129928,...,113.274761,-125.351730,-112.392606,84.511029,-0.973952,15.908779,-18.841565,-160.098300,-136.259174,87.473609
1,-49.893687,18.895526,-19.375540,-74.525859,107.439993,-54.700459,27.851507,155.527355,-4.301749,-16.740749,...,31.257570,46.658590,80.708687,35.561655,-55.801603,-1.804337,16.762459,-48.417262,-54.455946,7.580393
2,-47.602179,59.240664,-170.546812,37.933343,160.854868,-53.999917,51.403730,350.529876,-156.185388,79.324486,...,-21.841644,5.519694,49.528196,-8.714209,-75.047992,77.472366,68.381383,-202.809128,-209.031684,-2.645353
3,139.703089,99.907719,-117.146752,-195.251185,-93.167059,57.679924,18.830494,222.513734,-150.106029,53.864454,...,136.735054,-158.376765,72.018572,-74.121658,-13.490896,-94.343843,71.497790,88.893508,-181.188486,225.376829
4,2.701452,-129.127618,-210.715298,43.279201,167.406328,-233.408345,1.065302,344.294324,173.057927,-92.936881,...,53.214008,-102.245669,286.898936,-67.509254,-265.855143,155.566666,363.224012,-9.225736,48.108821,-69.294808


In [44]:
df_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-5.636833,80.486828,-251.079084,-26.783884,109.241614,61.187399,-75.228041,144.998795,-279.541222,10.113139,...,90.408938,-109.169522,-105.298152,94.436688,21.375816,-15.714312,-14.523175,-128.565776,-125.677323,89.598911
1,-111.078820,56.043106,-115.195678,-51.834606,90.433360,42.487261,35.498472,200.066768,43.483617,22.571563,...,63.994683,51.832590,103.229170,42.475588,-114.494393,41.895365,-91.993182,37.769108,-116.163856,80.441714
2,-46.931908,-2.988862,-156.264106,56.552071,58.780375,-23.859908,11.149397,283.233873,-116.120565,-6.824448,...,92.159742,-34.952615,48.003319,86.531880,-55.548438,116.778670,129.910394,-212.311876,-69.748311,180.982956
3,-174.706554,-474.474924,27.585909,-20.939406,674.584198,-133.671103,29.003045,275.769731,112.920334,-150.077855,...,-63.381199,70.248819,-234.656452,86.259855,180.287352,99.685409,382.765637,-82.232671,222.380742,36.688601
4,-10.961195,-4.420273,-120.162019,27.128121,99.285360,-82.145307,-31.274555,188.458215,-35.395586,39.561100,...,101.312017,-56.145697,110.729884,-53.758320,-152.819786,-51.701090,16.793719,-54.383640,-28.313034,-0.346219


In [45]:
df_q1["id"]=data["id"]
df_q2["id"]=data["id"]

In [46]:
%%time
final_df=data.merge(df_q1,on='id',how='left').merge(df_q2,on='id',how='left').drop(columns=['q1_feats_m','q2_feats_m'])

CPU times: user 476 ms, sys: 3.33 s, total: 3.81 s
Wall time: 10.3 s


In [47]:
final_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,...,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,1,1,66,57,...,90.408938,-109.169522,-105.298152,94.436688,21.375816,-15.714312,-14.523175,-128.565776,-125.677323,89.598911
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,4,1,51,88,...,63.994683,51.832590,103.229170,42.475588,-114.494393,41.895365,-91.993182,37.769108,-116.163856,80.441714
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1,1,73,59,...,92.159742,-34.952615,48.003319,86.531880,-55.548438,116.778670,129.910394,-212.311876,-69.748311,180.982956
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0,1,1,50,65,...,-63.381199,70.248819,-234.656452,86.259855,180.287352,99.685409,382.765637,-82.232671,222.380742,36.688601
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,3,1,76,39,...,101.312017,-56.145697,110.729884,-53.758320,-152.819786,-51.701090,16.793719,-54.383640,-28.313034,-0.346219


In [48]:
final_df.shape

(404290, 632)

In [57]:
final_df.to_csv("final_output_quora",index=False)